![resnet](https://github.com/user-attachments/assets/c85e0a97-7212-447c-b575-82aabb7d1b78)

- import libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

c:\Users\jayiu\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import shutil

shutil.rmtree('images')

In [3]:
import os
import requests

# 디렉토리 생성
try:
    os.makedirs("images/dogs", exist_ok=True)
    os.makedirs("images/cats", exist_ok=True)
except Exception as e:
    print(f"Error creating directories: {e}")

# 이미지 다운로드 함수
def download_image(url, folder):
    response = requests.get(url)
    if response.status_code == 200:
        filename = os.path.join(folder, url.split("/")[-1])
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {url} to {filename}")
    else:
        print(f"Failed to download {url}")

# 이미지 다운로드
dog_images = [
    "https://i.kinja-img.com/gawker-media/image/upload/s--WFkXeene--/c_scale,f_auto,fl_progressive,q_80,w_800/ol9ceoqxidudap8owlwn.jpg",
    "https://www.rspcansw.org.au/wp-content/uploads/2017/08/50_a-feature_dogs-and-puppies_mobile.jpg"
]

cat_images = [
    "https://www.catster.com/wp-content/uploads/2018/05/A-gray-cat-crying-looking-upset.jpg",
    "https://www.scarymommy.com/wp-content/uploads/2018/01/c1.jpg?w=700"
]

for url in dog_images:
    download_image(url, "images/dogs")

for url in cat_images:
    download_image(url, "images/cats")

Downloaded https://i.kinja-img.com/gawker-media/image/upload/s--WFkXeene--/c_scale,f_auto,fl_progressive,q_80,w_800/ol9ceoqxidudap8owlwn.jpg to images/dogs\ol9ceoqxidudap8owlwn.jpg
Downloaded https://www.rspcansw.org.au/wp-content/uploads/2017/08/50_a-feature_dogs-and-puppies_mobile.jpg to images/dogs\50_a-feature_dogs-and-puppies_mobile.jpg
Downloaded https://www.catster.com/wp-content/uploads/2018/05/A-gray-cat-crying-looking-upset.jpg to images/cats\A-gray-cat-crying-looking-upset.jpg
Failed to download https://www.scarymommy.com/wp-content/uploads/2018/01/c1.jpg?w=700


- hyperparameter

In [4]:
batch_size= 1
learning_rate = 0.0002
num_epoch = 100

- Data Loader

In [5]:
img_dir = './images'
img_data = dset.ImageFolder(img_dir, transforms.Compose([
                                     transforms.Resize(256),
                                     transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
]))
train_loader = data.DataLoader(img_data, batch_size = batch_size, shuffle = True, num_workers = 2)

- Model
  - Basic Block
    - 컨볼루션 연산과 활성화함수는 항상 붙어있음. 이를 함수로 만듬

In [6]:
def conv_block_1(in_dim, out_dim, act_fn, stride = 1):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size = 1, stride = stride),
        act_fn,
    )
    return model

def conv_block_3(in_dim, out_dim, act_fn):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size = 3, stride = 1, padding = 1),
        act_fn,
    )
    return model

- Model
  - Bottle Neck Module
    - 1x1 convolution -> 3x3 convolution -> 1x1 convolution
    - 실선은 크기가 변하지 않는 경우, 점선은 크기가 줄어드는 경우
    - down변수로 크기 감쇼 여부 표시, 조건문으로 경우의 수를 나눠서 구현
    - Skip-connection은 단순 더하기로 정의되어 있음 -> 피쳐맵의 크기를 일치시켜야 함
    - 차원을 맞춰주는 역할로 dim_equalizer 정의

In [7]:
class BottleNeck(nn.Module):
    def __init__(self, in_dim, mid_dim, out_dim, act_fn, down = False):
        super(BottleNeck, self).__init__()
        self.down = down
        
        if self.down:
            self.layer = nn.Sequential(
                conv_block_1(in_dim, mid_dim, act_fn, 2),
                conv_block_3(mid_dim, mid_dim, act_fn),
                conv_block_1(mid_dim, out_dim, act_fn),
            )
            self.downsample = nn.conv2d(in_dim, out_dim, 1, 2)
        else:
            self.layer = nn.Sequential(
                conv_block_1(in_dim, mid_dim, act_fn),
                conv_block_3(mid_dim, mid_dim, act_fn),
                conv_block_1(mid_dim, out_dim, act_fn),
            )
        
        self.dim_equalizer = nn.Conv2d(in_dim, out_dim, kernel_size = 1)
        
    def forward(self, x):
        if self.down:
            downsample = self.downsample(x)
            out = self.layer(x)
            out = out + downsample
        else:
            out = self.layer(x)
            if x.size() is not out.size():
                x = self.dim_equalizer(x)
            out = out + x
        return out

- ResNet Model

In [9]:
class ResNet(nn.Module):
    def __init__(self, base_dim, num_classes = 2):
        super(ResNet, self).__init__()
        self.act_fn = nn.ReLU()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3, base_dim, 7, 2, 3),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1),
        )
        self.layer_2 = nn.Sequential(
            BottleNeck(base_dim, base_dim, base_dim*4, self.act_fn),
            BottleNeck(base_dim*4, base_dim, base_dim*4, self.act_fn),
            BottleNeck(base_dim*4, base_dim, base_dim*4, self.act_fn, down = True),
        )
        self.layer_3 = nn.Sequential(
            BottleNeck(base_dim*4, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn, down = True),
        )
        self.layer_4 = nn.Sequential(
            BottleNeck(base_dim*8, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn, down = True),
        )
        self.layer_5 = nn.Sequential(
            BottleNeck(base_dim*16, base_dim*8, base_dim*32, self.act_fn),
            BottleNeck(base_dim*32, base_dim*8, base_dim*32, self.act_fn),
            BottleNeck(base_dim*32, base_dim*8, base_dim*32, self.act_fn),
        )
        self.avgpool = nn.AvgPool2d(7, 1)
        self.fc_layer = nn.Linear(base_dim*32, num_classes)
        
    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        out = self.layer_5(out)
        out - self.avgpool(out)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        
        return out